# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

In [47]:
foods = [:QP, :MD, :BM, :FF, :MC, :FR, :SM, :M1, :OJ]
# Word to the wise---Julia symbols cannot start with a number (M1)
nutrients = [:Prot, :VitA, :VitC, :Calc, :Iron, :Cals, :Carb]

cost = Dict(zip(foods,[1.84,2.19,1.84,1.44,2.29,0.77,1.29,0.6,0.72]))
required = Dict(zip(nutrients,[55,100,100,100,100,2000,350]))
using NamedArrays
A = [
    28 24 25 14 31 3 15 9 1
    15 15 6 2 8 0 4 10 2
    6 10 2 0 15 15 0 4 120
    30 20 25 15 15 0 20 30 2
    20 20 20 10 8 2 15 0 2
    510 370 500 370 400 220 345 110 80
    34 33 42 38 42 26 27 12 20
]
A_NA = NamedArray(A, (nutrients,foods),("Nutrients","Menu Items")) ;





In [48]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Min, sum(cost[j]*x[j] for j in foods))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can try this
set_silent(m)
optimize!(m)

using Formatting
printfmtln("Minimum cost menu \${:2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:2f} of menu item {2}", value(x[j]), j)
    end
end



Minimum cost menu $14.855738: 
Eat 4.385246 of menu item :QP
Eat 6.147541 of menu item :FR
Eat 3.422131 of menu item :M1


Let's show how to read it from a spreadsheet
(mcdonalds.csv)

DataFrames is like Pandas in Python: https://dataframes.juliadata.org/stable/


In [45]:
using DataFrames, CSV, NamedArrays
df = CSV.read("mcdonalds.csv", DataFrame, delim =',') 

# Foods are column names starting at the 3rd
foods = propertynames(df)[3:end]

# Nutrients are the 
nutrients = convert(Array, df[2:end,1])

# Cost
cost = Dict(zip(foods,df[1,3:end]))

required = Dict(zip(nutrients,df[2:end,2]))

# Matrix of data 
A = Matrix(df[2:end,3:end])

A_NA = NamedArray(A, (nutrients,foods), ("Nutrients", "Foods")) ;

# This should solve with the exact same model


7×9 Named Matrix{Float64}
    Nutrients ╲ Foods │   …  
──────────────────────┼──────
String15("Protein")   │   …  
String15("Vitamin A") │      
String15("Vitamin C") │      
String15("Calcium")   │      
String15("Iron")      │      
String15("Calories")  │      
String15("Carbs")     │   …  

In [46]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Min, sum(cost[j]*x[j] for j in foods))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can try this
set_silent(m)
optimize!(m)

using Formatting
printfmtln("Minimum cost menu \${:2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:2f} of menu item {2}", value(x[j]), j)
    end
end

Minimum cost menu $14.855738: 
Eat 4.385246 of menu item Symbol("Quarter Pounder")
Eat 6.147541 of menu item Symbol("Small Fries")
Eat 3.422131 of menu item Symbol("1% Milk")
